# Reversible Network Reconnection Model

Attempt at implementing the RNR model as described in Okuda et al. 2012:

>_Reversible network reconnection model for simulating large deformation in dynamic tissue morphogenesis_, Satoru Okuda, Yasuhiro Inoue, Mototsugu Eiraku, Yoshiki Sasai and Taiji Adachi Biomech Model Mechanobiol (2013) 12:627–644 [DOI 10.1007/s10237-012-0430-7](http://link.springer.com/article/10.1007%2Fs10237-012-0430-7)


The authors give 4 conditions which we detail and implement bellow.




## Condition 1 - Center of a face

The center of a face is defined by the average position of the face's edges midpoints, weighted by their lengths.

$$ \mathbf{r}_{\alpha} = \frac{\sum_{ij\alpha}\ell_{ij} (\mathbf{r}_i + \mathbf{r}_j)/2}{\sum_{ij\alpha}\ell_{ij}}$$


In [1]:
import numpy as np, pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

from tyssue import Epithelium
from tyssue import BulkGeometry
from tyssue import Sheet
from tyssue.config.geometry import bulk_spec

from tyssue.core.generation import three_faces_sheet
from tyssue.core.generation import extrude
from tyssue.draw.threejs_draw import view_3js
from tyssue.draw.plt_draw import quick_edge_draw

class RNRGeometry(BulkGeometry):
    
    @staticmethod
    def update_centroid(eptm):

        srce_pos = eptm.upcast_srce(eptm.vert_df[eptm.coords])
        trgt_pos = eptm.upcast_trgt(eptm.vert_df[eptm.coords])
        mid_pos = (srce_pos + trgt_pos)/2
        eptm.face_df[eptm.coords] = (eptm.sum_face(mid_pos).values /
                                     eptm.face_df['perimeter'].values[:, np.newaxis])
        srce_pos['cell'] = eptm.edge_df['cell']
        eptm.cell_df[eptm.coords] = srce_pos.groupby('cell').mean()

sheet = Sheet.planar_sheet_3d('sheet', 5, 5, 1, 1)
sheet.sanitize()
datasets = extrude(sheet.datasets, method='translation')

eptm = Epithelium('20faces_3D', datasets, bulk_spec())
#BulkGeometry.update_all(eptm)
RNRGeometry.update_all(eptm)



In [2]:
eptm.edge_df.head()

,srce,dx,trgt,dy,length,face,nz,nx,ny,dz,segment,cell,sub_area,sub_vol
edge,,,,,,,,,,,,,,
0,2,-1.25,0,-0.625,1.397542,4,0.589709,0.0,-0.0,0.0,apical,4,0.294855,0.049142
1,1,0.00,2,-0.750,0.750000,8,0.829475,0.0,0.0,0.0,apical,8,0.414737,0.069123
2,0,1.25,4,-0.625,1.397542,4,0.950928,0.0,0.0,0.0,apical,4,0.475464,0.079244
3,4,0.00,3,-0.750,0.750000,0,0.829475,0.0,0.0,0.0,apical,0,0.414737,0.069123
4,2,0.50,5,-0.250,0.559017,8,1.372941,0.0,0.0,0.0,apical,8,0.686471,0.114412


In [3]:
render, line = view_3js(eptm)
display(render)

## I→H transition

![IH transition in a bulk and a monolayer](../data/png/IH_transition.png)



In [10]:
srce_orbits = eptm.get_orbits('srce', 'face')

In [13]:
def IH_transition(eptm, e_1011):
    
    v_pairs, common_cells = get_vertex_pairs(eptm, e_1011)
    
    
    try:
        (v1, v4), (v2, v5), (v3, v6) = v_pairs
    except ValueError:
        print('Edge {} is not a valid junction to'
              ' perform IH transition on, aborting'.format(e_1011))
        return
    new_vs = eptm.vert_df.loc[[v1, v2, v3]].copy()
    eptm.vert_df.append(new_vs, ignore_index=True)
    v7, v8, v9 = eptm.vert_df.index[-3:]
    for pair, new in zip(v_pairs, (v7, v8, v9)):
        
        va, vb = pair
        e_a10s = eptm.edge_df[(eptm.edge_df['srce'] == va) &
                              (eptm.edge_df['trgt'] == v10)].index
        eptm.edge_df.loc[e_a10s, 'trgt'] = new

        e_10as = eptm.edge_df[(eptm.edge_df['srce'] == v10) &
                              (eptm.edge_df['trgt'] == va)].index
        eptm.edge_df.loc[e_10as, 'srce'] = new
        
        e_b11s = eptm.edge_df[(eptm.edge_df['srce'] == vb) &
                              (eptm.edge_df['trgt'] == v11)].index
        eptm.edge_df.loc[e_b11s, 'trgt'] = new

        e_11bs = eptm.edge_df[(eptm.edge_df['srce'] == v11) &
                              (eptm.edge_df['trgt'] == vb)].index
        eptm.edge_df.loc[e_11bs, 'srce'] = new
        
    
def get_vertex_pairs(eptm, e_1011):
    
    srce_orbits = eptm.get_orbits('srce', 'cell')
        
    v10, v11 = eptm.edge_df.loc[e_1011, ['srce', 'trgt']]
    v10_out = set(eptm.edge_df[eptm.edge_df['srce']==v10]['trgt']) - {v11}
    cells_123 = {v: set(srce_orbits.loc[v])
                 for v in v10_out}
    
    v11_out = set(eptm.edge_df[eptm.edge_df['srce']==v11]['trgt']) - {v10}
    cells_456 = {v: set(srce_orbits.loc[v])
                 for v in v11_out}
    v_pairs = []
    common_cells = []
    for vi in v10_out:
        for vj in v11_out:
            common_cell = cells_123[vi].intersection(cells_456[vj])
            if len(common_cell) > 0:
                v_pairs.append((vi, vj))
                common_cells.append(common_cell)
                break
        else:
            raise ValueError('No vertex for vertex %i' % vi)
    return v_pairs, common_cells
    
    

In [14]:
edge = 56
get_vertex_pairs(eptm, edge)

([(24, 66), (33, 66), (69, 66)], [{11}, {10}, {10, 11}])